<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/sk_whisper_small_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7em609k1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7em609k1
  Resolved https://github.com/huggingface/transformers to commit ebfdb9ca62205279d5019ef1403877461b3b2da4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [81]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [82]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [83]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "sk", split="test")
#column_name = "audio, normalized_text"

In [84]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 604
})

In [85]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [86]:
model = whisper.load_model("small")

In [87]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [88]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [89]:
text

,0
0,ale rád by som zdôraznil nepripomínajme si len...
1,je nezodpovedné aby počas svetovej hospodárske...
2,vyzývam komisára timmermansa aby pripravil spr...
3,je načase pozrieť sa na spoločnú poľnohospodár...
4,na druhej strane sa mi nepáči natláčanie a nič...
...,...
599,pandémia nám však ukázala že naša spoločnosť m...
600,reálny vývoj totiž jednoznačne potvrdzuje že j...
601,a toto je dámy a páni veľmi dobrá správa pre s...
602,je ale dobré že je v tom zamontovaný demokrati...


In [90]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [91]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,ale rád by som zdôraznil nepripomínajme si len...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000762939453125, -0.000518798828125, 0.000...",16000
1,male,je nezodpovedné aby počas svetovej hospodárske...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003021240234375, -0.01214599609375, -0.011...",16000
2,male,vyzývam komisára timmermansa aby pripravil spr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003875732421875, 0.03155517578125, 0.001922...",16000
3,female,je načase pozrieť sa na spoločnú poľnohospodár...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0234375, 0.02197265625, 0.024993896484375, ...",16000
4,male,na druhej strane sa mi nepáči natláčanie a nič...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.00054931640625, -0.0010...",16000
...,...,...,...,...,...
599,female,pandémia nám však ukázala že naša spoločnosť m...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000701904296875, 0.0, -0.00091552734375, -0...",16000
600,male,reálny vývoj totiž jednoznačne potvrdzuje že j...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00201416015625, -0.00408935546875, -0.00698...",16000
601,male,a toto je dámy a páni veľmi dobrá správa pre s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.07574462890625, 0.08111572265625, 0.0905761...",16000
602,male,je ale dobré že je v tom zamontovaný demokrati...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.015777587890625, -0.004364013671875, 0.009...",16000


In [92]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,ale rád by som zdôraznil nepripomínajme si len...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000762939453125, -0.000518798828125, 0.000...",16000
1,male,je nezodpovedné aby počas svetovej hospodárske...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003021240234375, -0.01214599609375, -0.011...",16000
2,male,vyzývam komisára timmermansa aby pripravil spr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003875732421875, 0.03155517578125, 0.001922...",16000
4,male,na druhej strane sa mi nepáči natláčanie a nič...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.00054931640625, -0.0010...",16000
5,male,a rovnako ako ostatných štátov sa to dotýka aj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003448486328125, 0.006500244140625, 0.00289...",16000
...,...,...,...,...,...
598,male,preto som rád že aj európsky parlament sa prip...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000885009765625, -0.000152587890625, 0.001...",16000
600,male,reálny vývoj totiž jednoznačne potvrdzuje že j...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00201416015625, -0.00408935546875, -0.00698...",16000
601,male,a toto je dámy a páni veľmi dobrá správa pre s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.07574462890625, 0.08111572265625, 0.0905761...",16000
602,male,je ale dobré že je v tom zamontovaný demokrati...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.015777587890625, -0.004364013671875, 0.009...",16000


In [93]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
3,female,je načase pozrieť sa na spoločnú poľnohospodár...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0234375, 0.02197265625, 0.024993896484375, ...",16000
12,female,je veľmi dôležité aby sme našich spoluobčanov ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009124755859375, 0.001312255859375, 0.0131...",16000
13,female,aj jedna tragická nehoda je veľa a preto veľmi...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.014129638671875, 0.003509521484375, -0.0007...",16000
21,female,na druhej strane sme sa však v našej správe us...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.00146484375, -0.00155639648...",16000
22,female,správa o integrovanom prístupe medzi ženami a ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00048828125, -0.000457763671875, -0.000244...",16000
...,...,...,...,...,...
590,female,komisia pripravila akési praktické príručky pr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00042724609375, 0.000213623046875, -3.05175...",16000
591,female,tento program pozitívne ovplyvnil za tridsať r...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003997802734375, -0.004974365234375, -0.004...",16000
594,female,vyhodnotenie by malo určite vyššiu výpovednú h...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000885009765625, -0.000823974609375, 0.0, ...",16000
595,female,v období keď v členských štátoch narastá euros...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 3.0517578125e-05, -0.0003...",16000


In [94]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="sk", fp16=False)
    return result["text"]

In [95]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [96]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [97]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.47368421052631576, Precision: 0.631578947368421, Recall: 0.7058823529411765, F1: 0.6666666666666667
Processed 2 files, WER: 0.7142857142857143, Precision: 0.35714285714285715, Recall: 0.35714285714285715, F1: 0.35714285714285715
Processed 3 files, WER: 0.45454545454545453, Precision: 0.6190476190476191, Recall: 0.5909090909090909, F1: 0.6046511627906977
Processed 4 files, WER: 0.65, Precision: 0.45, Recall: 0.45, F1: 0.45
Processed 5 files, WER: 0.6, Precision: 0.4444444444444444, Recall: 0.4, F1: 0.4210526315789474
Processed 6 files, WER: 0.41935483870967744, Precision: 0.6071428571428571, Recall: 0.5862068965517241, F1: 0.5964912280701754
Processed 7 files, WER: 0.2, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 8 files, WER: 0.6470588235294118, Precision: 0.42105263157894735, Recall: 0.5333333333333333, F1: 0.47058823529411764
Processed 9 files, WER: 0.5294117647058824, Precision: 0.4375, Recall: 0.4375, F1: 0.4375
Processed 10 files, WER: 0

In [99]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.5, Precision: 0.5555555555555556, Recall: 0.5, F1: 0.5263157894736842
Processed 2 files, WER: 0.5, Precision: 0.5909090909090909, Recall: 0.5909090909090909, F1: 0.5909090909090909
Processed 3 files, WER: 0.5909090909090909, Precision: 0.45454545454545453, Recall: 0.45454545454545453, F1: 0.45454545454545453
Processed 4 files, WER: 0.6666666666666666, Precision: 0.4090909090909091, Recall: 0.42857142857142855, F1: 0.4186046511627907
Processed 5 files, WER: 0.4864864864864865, Precision: 0.6428571428571429, Recall: 0.5142857142857142, F1: 0.5714285714285714
Processed 6 files, WER: 0.55, Precision: 0.5238095238095238, Recall: 0.55, F1: 0.5365853658536585
Processed 7 files, WER: 0.43243243243243246, Precision: 0.5483870967741935, Recall: 0.53125, F1: 0.5396825396825397
Processed 8 files, WER: 0.5833333333333334, Precision: 0.4090909090909091, Recall: 0.391304347826087, F1: 0.4
Processed 9 files, WER: 0.3125, Precision: 0.6875, Recall: 0.6875, F1: 0.6875
Processed